In [1]:
import sys
from pathlib import Path
parent_dir = Path('.').resolve().parent
sys.path.insert(0, str(parent_dir))

import numpy as np
import pandas as pd
import math
import statistics
import random
import importlib

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor

import math_expressions as mexpr
importlib.reload(mexpr)

import methods
importlib.reload(methods)

<module 'methods' from '/Users/preetkaria/Desktop/RA/time_to_leave/methods.py'>

In [2]:
df = pd.read_csv('../data/gen_data.csv')

In [3]:
df.head()

,alpha,beta,N,n,h,c,mean_n,std_n,alpha_hat,beta_hat,u_star,u_star_hat,z,u,optimal_cost,actual_cost,intervals_str
0,6,2,4,5,0.666667,20,12.443551,5.159124,6,2.138984,38.093382,41.275446,0.922907,27.047200,20.000000,20.000000,19.330937006565275_6.304903371950173_10.358245...
1,2,4,4,5,0.066667,30,8.348155,2.476622,11,0.734732,9.748366,20.340700,0.479254,38.173921,1.895037,1.188881,6.42533611970204_7.267416045547929_12.11888158...
2,8,6,3,5,0.666667,10,48.678803,16.972756,8,5.917862,216.726649,211.572577,1.024361,111.995762,10.000000,10.000000,20.79192862252269_56.65662483403159_64.4600453...
3,8,6,5,5,0.666667,20,39.532671,11.340199,12,3.253009,269.480306,195.829568,1.376096,228.044282,20.000000,21.476476,23.400532927359993_39.20904124992205_35.616708...
4,4,6,5,5,0.133333,30,21.104856,10.097073,4,4.830684,77.248992,60.195813,1.283295,123.206460,6.127662,8.401420,29.600895845585907_4.3327499423800635_23.08522...


In [4]:
X = df[['N', 'n', 'h', 'c', 'mean_n', 'std_n', 'alpha_hat', 'beta_hat', 'u_star_hat']]
y = df['z']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=50)


In [5]:
model = LinearRegression()

model.fit(X_train, y_train)
display(model.score(X_train, y_train))
display(model.coef_)

1.0

array([ 0.00025777,  0.        ,  0.00022004, -0.00338091,  0.00467295,
        0.00287128, -0.00349487,  0.00153059,  0.01735143])

In [6]:
test_output = pd.DataFrame(model.predict(X_test), index=X_test.index, columns=['predicted_z'])
test_df = test_output.merge(df, left_index=True, right_index=True)
test_df['predicted_u_star'] = test_df['predicted_z'] * test_df['u_star_hat']
test_df['actual_cost'] = test_df.apply(lambda row: methods.cal_cost(row['c'], row['h'], row['u'], row['predicted_u_star']), axis=1)
test_df.head()

,predicted_z,alpha,beta,N,n,h,c,mean_n,std_n,alpha_hat,beta_hat,u_star,u_star_hat,z,u,optimal_cost,actual_cost,intervals_str,predicted_u_star
3,3.729969,8,6,5,5,0.666667,20,39.532671,11.340199,12,3.253009,269.480306,195.829568,1.376096,228.044282,20.0,20,23.400532927359993_39.20904124992205_35.616708...,730.438272
2,4.113397,8,6,3,5,0.666667,10,48.678803,16.972756,8,5.917862,216.726649,211.572577,1.024361,111.995762,10.0,10,20.79192862252269_56.65662483403159_64.4600453...,870.281989


In [7]:
methods.plot_plotly(data=test_df['actual_cost'], data_label='Actual Cost')
methods.plot_plotly(data=test_df['optimal_cost'], data_label='Optimal Cost')
methods.multi_plot_plotly(data=[test_df['optimal_cost'], test_df['actual_cost']], data_label=['Optimal Cost', 'Actual Cost'])
